In [7]:
import csv
import random
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
#确定特征属性，并获取训练样本
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    #print(dataset)
    #print(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset
    
filename = 'pima-indians-diabetes.data.csv'
dataset = loadCsv(filename)
#print(dataset)
#print('Loaded data file {0} with {1} rows.'.format(filename, len(dataset)))
dataset = np.reshape(dataset, (len(dataset), 9))
X = dataset[:, 0:8]
Y = dataset[:, 8].reshape(-1, 1)
#print(train_Y.shape)
#print(train_set)
#print(train_X)
#print(train_X.shape)


In [9]:
#数据预处理，分隔训练集和测试集
def splitDataSet(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def splitDataSet_sklearn(X, Y, splitRatio):
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=splitRatio, random_state=33)
    return [train_X, test_X, train_Y, test_Y]

train_X, test_X, train_y, test_y = splitDataSet_sklearn(X, Y, 0.33)
print(train_X.shape)
print(test_X.shape)


(514, 8)
(254, 8)


In [10]:
#按类别划分数据，（为每个数据计算p(y)）
def separateByClass(dataset):
    separated ={}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def separateByClass_sec(x, y):
    separated = {}
    for i in range(len(x)):
        vector = x[i]
        label = y[i, 0]
        if (label not in separated):
            separated[label] = []
        separated[label].append(vector)
    return separated

sepa_x = separateByClass_sec(train_X, train_y)
#print(sepa_x)

In [11]:
#辅助函数
def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)



In [12]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    #del summaries[-1]
    return summaries

#dataset = [[1, 20, 0], [2, 21, 1]]
#summary = summarize(dataset)
#print('Attribute summaries: {0}'.format(summary))

#char_X = summarize(train_X)

In [13]:
#计算各个类别的均值和方差，用于估计各个类别的高斯分布函数时使用（连续分布）
#离散时则直接统计各个特征数目，计算其与该类对应值比。
def summarizeByClass_sec(x, y):
    separated = separateByClass_sec(x, y)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

#summarizeByClass_sec(train_X, train_y)


In [14]:
#对每个特征属性计算所划分的条件概率
#获取高斯分布的概率密度函数
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean, 2)/(2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent



In [17]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

inputVector = [1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, '?']
probabilities = calculateClassProbabilities(summarizeByClass_sec(train_X, train_y), inputVector)
print(probabilities)

{0.0: 2.8117174224713984e-135, 1.0: 3.127936464682264e-100}


In [19]:
#选取最大的概率作为预测值
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [ ]:
#简单计算预测准确率
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0